# Apply OE

Apply OE is the built-in end-to-end atmospheric correction pipeline, and is the easiest way to run Isofit. Apply OE is run via a command line interface (CLI) tool accessed with a terminal call:

```bash
isofit apply_oe --help
```

The function docstring is printed when this command is run within a terminal window, and acts as a guide for how to run the function.

In [ ]:
from isofit.utils.apply_oe import apply_oe

print(apply_oe.__doc__)

The Apply OE function can leverage a large number of input parameters, but most are optional. The important inputs are the non-optional arguments:

`INPUT_RADIANCE` `INPUT_LOC` `INPUT_OBS` `WORKING_DIRECTORY` `SENSOR` and `--surface_path`.

which must be entered in the specified order (except `--surface_path` which can be added at any point but still required). Descriptions of each are found in the docstring printed above. It is important to note that the `INPUT_RADIANCE`, `INPUT_LOC`, and `INPUT_OBS` are ENVI raster data formats that must be at the same row-column dimensions. The `--surface_path` points Isofit torwards the constructed prior distribution file for surface reflectance. Optional arguments are denoted by the '--' in their name, e.g.  `--modtran_path`, `--pressure_elevation`. It is important to note that the default radiative transfer engine (RTE) is currently set to Modtran. You must specify an `--emulator_path` without a Modtran installation.

## How do you run Apply OE?

The script is run via the CLI. For example:

> `IC=$(isofit path examples)/image_cube/small` sets the environment variable `IC` to the output of the command `isofit path examples/image_cube/small`. This is just used to reuse the same output for multiple arguments.
> Depending on the user's isofit installation environment, this path may vary. This command ensures the correct path is retrieved

```bash
  IC=$(isofit path examples)/image_cube/small EX=$(isofit path examples)/image_cube/small \
  isofit apply_oe \
  $IC/ang20170323t202244_rdn_7000-7010 \
  $IC/ang20170323t202244_loc_7000-7010 \
  $IC/ang20170323t202244_obs_7000-7010 \
  $EX \
  ang \
  --surface_path $EX/configs/surface.json \
  --emulator_base $(isofit path srtmnet --key file) \
  --n_cores 10 \
  --presolve \
```

Here,

```
$IC/ang20170323t202244_rdn_7000-7010
$IC/ang20170323t202244_loc_7000-7010
$IC/ang20170323t202244_obs_7000-7010
```

are the radiance, location, and observational geometry files respectively. The `\` tells the CLI call to expect a multi-line input. The remaining two requried parameters are `ang`, the sensor designation (AVIRIS-NG), and the `--surface_path` pointing to the surface configuration file which will be built at runtime.

The remaining arguments set Apply OE to run with:
1) `--emulator_base` points isofit to the location of the sRTMnet emulator to usse as the radiative transfer engine (RTE)
2) `--n_cores = 10` CPU cores
3) The `--presolve` algorithm to narrow down the water vapor retrievals

In [ ]:
# Alternatively, you can programatically call apply_oe. If you executed apply_oe via the command line above, do not run this
# We use the env object to retrieve the actual location of the ISOFIT extra dependencies instead of assuming where they could be
import os
import shutil

from isofit.utils.apply_oe import apply_oe
from isofit.data import env

output = env.path("examples", "image_cube", "small")

# Cleanup any previous runs; comment this out if you want to preserve a previous run's output
if (o := output / "output").exists():
    shutil.rmtree(o)

apply_oe(
    input_radiance    = str(env.path("examples", "image_cube", "small", "ang20170323t202244_rdn_7000-7010")),
    input_loc         = str(env.path("examples", "image_cube", "small", "ang20170323t202244_loc_7000-7010")),
    input_obs         = str(env.path("examples", "image_cube", "small", "ang20170323t202244_obs_7000-7010")),
    working_directory = str(output),
    sensor            = "ang",
    surface_path      = str(env.path("examples", "image_cube", "small", "configs", "surface.json")),
    emulator_base     = str(env.path("srtmnet", key="srtmnet.file")),
    presolve          = True,
    n_cores           = os.cpu_count(),
)

We can examine both the inputs and outputs of Apply OE with this run call:

In [ ]:
# Common imports
import os
from pathlib import Path

from spectral import envi
from matplotlib import pyplot as plt
import numpy as np

from isofit.core.common import envi_header
from isofit.data import env

In [ ]:
# Load the input files
rdn_path = env.path("examples", "image_cube", "small", "ang20170323t202244_rdn_7000-7010")
loc_path = env.path("examples", "image_cube", "small", "ang20170323t202244_loc_7000-7010")
obs_path = env.path("examples", "image_cube", "small", "ang20170323t202244_obs_7000-7010")

rdn = envi.open(envi_header(str(rdn_path)))
loc = envi.open(envi_header(str(loc_path)))
obs = envi.open(envi_header(str(obs_path)))

rdn_im = rdn.open_memmap(interleave='bip')
loc_im = loc.open_memmap(interleave='bip')
obs_im = obs.open_memmap(interleave='bip')

In [ ]:
# Print the bands of the input files
print('Band names in the location file:')
[print(f"{i}") for i in loc.metadata['band names']]

print()
print('Band names in the observational geometry file:')
temp = [print(f"{i}") for i in obs.metadata['band names']]

In [ ]:
# Plot the input data
normalize = lambda x, vmin, vmax:  (x - vmin) / (vmax - vmin)
bands = [55, 35, 15]

fig, axs = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(13, 4))
plot = axs[0].imshow(normalize(rdn_im[..., bands], 0, 15))
plot = axs[1].imshow(loc_im[..., 0])
plot = axs[2].imshow(obs_im[..., 4])

title = axs[0].set_title('Radiance (RGB)')
title = axs[1].set_title('Longitude (WGS-84)')
title = axs[2].set_title('Solar zenith angle (Deg)')

The input image doesn't look like much because this is just a 10x10 pixel example. However we see per-pixel spectral variation in the radiance RGBs, and systematic variation in the location and geometric variables.

In [ ]:
# Load the output files
rfl_path = env.path("examples", "image_cube", "small", "output", "ang20170323t202244_rfl")
state_path = env.path("examples", "image_cube", "small", "output", "ang20170323t202244_state")
uncert_path = env.path("examples", "image_cube", "small", "output", "ang20170323t202244_uncert")

rfl = envi.open(envi_header(str(rfl_path)))
state = envi.open(envi_header(str(state_path)))
uncert = envi.open(envi_header(str(uncert_path)))

rfl_im = rfl.open_memmap(interleave='bip')
state_im = state.open_memmap(interleave='bip')
uncert_im = uncert.open_memmap(interleave='bip')

print(f'Shape of the _rfl file: {rfl_im.shape}')
print(f'Shape of the _state file: {state_im.shape}')
print(f'Shape of the _uncert file: {uncert_im.shape}')

The difference between the `_rfl` file and the `_state` file is that the `_rfl` file only contains the solutions for surface reflectance variables. Here, the AVIRIS-NG image contains 425 wavelength bands. As a result, the `_rfl` contains 425 bands. The `_state` and `_uncert` files contain the surface reflectance solutions and uncertainty calculated as the standard deviation of the posterior distributions for the 425 wavelength bands and for non-reflectance statevector elements; here, aerosol optical depth (AOD) and water vapor (H2O).

In [ ]:
# Plot the output data
normalize = lambda x, vmin, vmax:  (x - vmin) / (vmax - vmin)
bands = [55, 35, 15]

fig, axs = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(8, 8))
axs = np.ravel(axs)
plot = axs[0].imshow(normalize(rfl_im[..., bands], 0, 0.3))
uncert = axs[1].imshow(uncert_im[..., 55])
aod = axs[2].imshow(state_im[..., -2])
h2o = axs[3].imshow(state_im[..., -1])

plt.colorbar(uncert)
plt.colorbar(aod)
plt.colorbar(h2o)

title = axs[0].set_title('Reflectance')
title = axs[1].set_title('Uncertainty (at 650 nm)')
title = axs[2].set_title('AOD')
title = axs[3].set_title('H2O')

## The Analytical Line

In the example above, we ran Isofit using the full optimal estimation (OE) on each pixel independently. Computationally, this amounts to iterating through each row-column pair to solve for the full state-vector (427 variables in the above case). However for large images, this demands long run-times, and ignores the principle that some state-vector elements, namely the atmospheric variables like AOD and H2O, should not vary from one pixel to another, but rather should be spatially smooth, and only vary over multi-pixel length scales [Link to the relevent paper](https://www.sciencedirect.com/science/article/pii/S0034425723004534/).

The analytical line and empirical line algorithms leverage the assumption of a spatially smooth atmosphere to decrease run times by a factor of 10. Currently, we suggest using the analytical line algorithm and not the empirical line algorithm.

See the following CLI call to run Apply OE with the analytical line algorithm:

```bash
  IC=$(isofit path examples)/image_cube/medium EX=$(isofit path examples)/image_cube/medium \
  isofit apply_oe \
  $IC/ang20170323t202244_rdn_7k-8k \
  $IC/ang20170323t202244_loc_7k-8k \
  $IC/ang20170323t202244_obs_7k-8k \
  $EX \
  ang \
  --surface_path $EX/configs/surface.json \
  --emulator_base $(isofit path srtmnet --key file) \
  --n_cores 10 \
  --presolve \
  --analytical_line \
  --segmentation_size 50 \
  --log_file $EX/log.txt
```

Most of the input parameters are identical to the per-pixel application above. However, we've added a `--log_file`, the `--analytical_line` flag, and a `--segmentation_size`. The `--log_file` points the program to write a text file to print logging statements during run time. The `--analytical_line` flag tells Isofit to use the analytical line algorithm.

A simple overview for the anylitical line algorithm:

1) All three input files are "segmented" into superpixel blocks using the SLIC algorithm. The  `--segmentation_size` value sets the number of pixels that each superpixel contains.

2) At the superpixel resolution, Isofit solves for the OE solutions, which provides both surface and atmospheric state variables.

3) Atmospheric state variables are spatially interpolated to full image resolution. The spatial interpolation uses the Apply OE parameters `--num_neighbors` and `--atm_sigma`.

4) With a fixed atmosphere, we leverage a closed form solution for surface state elements that allows for a solution convergence in a single iteration.

In [ ]:
# Alternatively, you can programatically call apply_oe. If you executed apply_oe via the command line above, do not run this
# We use the env object to retrieve the actual location of the ISOFIT extra dependencies instead of assuming where they could be
import os
import shutil

from isofit.utils.apply_oe import apply_oe
from isofit.data import env

output = env.path("examples", "image_cube", "medium")

# Cleanup any previous runs; comment this out if you want to preserve a previous run's output
if (o := output / "output").exists():
    shutil.rmtree(o)

apply_oe(
    input_radiance    = str(env.path("examples", "image_cube", "medium", "ang20170323t202244_rdn_7000-7010")),
    input_loc         = str(env.path("examples", "image_cube", "medium", "ang20170323t202244_loc_7000-7010")),
    input_obs         = str(env.path("examples", "image_cube", "medium", "ang20170323t202244_obs_7000-7010")),
    working_directory = str(output),
    sensor            = "ang",
    surface_path      = str(env.path("examples", "image_cube", "medium", "configs", "surface.json")),
    emulator_base     = str(env.path("srtmnet", key="srtmnet.file")),
    presolve          = True,
    n_cores           = os.cpu_count(),
    analytical_line   = True,
    segmentation_size = 50,
)

We can visualize what the segmentation is doing by leveraging the `isofit reconstruct_subs` CLI command:

```bash
EX=$(isofit path examples)/image_cube/medium isofit reconstruct_subs \
    $EX/input/ang20170323t202244_subs_rdn \
    $EX/input/ang20170323t202244_subs_recon_rdn \
    $EX/output/ang20170323t202244_lbl

EX=$(isofit path examples)/image_cube/medium isofit reconstruct_subs \
    $EX/input/ang20170323t202244_subs_loc \
    $EX/input/ang20170323t202244_subs_recon_loc \
    $EX/output/ang20170323t202244_lbl

EX=$(isofit path examples)/image_cube/medium isofit reconstruct_subs \
    $EX/input/ang20170323t202244_subs_obs \
    $EX/input/ang20170323t202244_subs_recon_obs \
    $EX/output/ang20170323t202244_lbl
```

In [ ]:
from isofit.utils.reconstruct import reconstruct_subs

for product in ("rdn", "loc", "obs"):
    reconstruct_subs(
        input_subs_path  = str(env.path("examples", "image_cube", "medium", "input", f"ang20170323t202244_subs_{product}")),
        output_path      = str(env.path("examples", "image_cube", "medium", "input", f"ang20170323t202244_subs_recon_{product}")),
        lbl_working_path = str(env.path("examples", "image_cube", "medium", "output", "ang20170323t202244_lbl")),
    )

Visualizing the input data:

In [ ]:
# Plotting the input data
rdn_path = env.path("examples", "image_cube", "medium", "ang20170323t202244_rdn_7000-7010")
loc_path = env.path("examples", "image_cube", "medium", "ang20170323t202244_loc_7000-7010")
obs_path = env.path("examples", "image_cube", "medium", "ang20170323t202244_obs_7000-7010")

subs_rdn_path = env.path("examples", "image_cube", "medium", "input", "ang20170323t202244_subs_recon_rdn")
subs_loc_path = env.path("examples", "image_cube", "medium", "input", "ang20170323t202244_subs_recon_loc")
subs_obs_path = env.path("examples", "image_cube", "medium", "input", "ang20170323t202244_subs_recon_obs")

rdn = envi.open(envi_header(str(rdn_path)))
loc = envi.open(envi_header(str(loc_path)))
obs = envi.open(envi_header(str(obs_path)))

subs_rdn = envi.open(envi_header(str(subs_rdn_path)))
subs_loc = envi.open(envi_header(str(subs_loc_path)))
subs_obs = envi.open(envi_header(str(subs_obs_path)))

rdn_im = rdn.open_memmap(interleave='bip')
loc_im = loc.open_memmap(interleave='bip')
obs_im = obs.open_memmap(interleave='bip')

subs_rdn_im = subs_rdn.open_memmap(interleave='bip')
subs_loc_im = subs_loc.open_memmap(interleave='bip')
subs_obs_im = subs_obs.open_memmap(interleave='bip')

fig, axs = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(10, 10))
axs = np.ravel(axs)

axs[0].imshow(normalize(rdn_im[0:400, :, bands], 0, 15))
axs[1].imshow(loc[0:400, :, 0])
axs[2].imshow(obs[0:400, :, 4])

axs[3].imshow(normalize(subs_rdn_im[0:400, :, bands], 0, 15))
axs[4].imshow(subs_loc[0:400, :, 0])
axs[5].imshow(subs_obs[0:400, :, 4])

axs[0].set_title('Radiance')
axs[1].set_title('Longitude')
axs[2].set_title('Elevation')

axs[0].set_ylabel('Full resolution')
axs[3].set_ylabel('Superpixel resolution')
plt.show()

We can examine the results from Apply OE after the OE solutions on the superpixels:

In [ ]:
wl = np.array(rdn.metadata['wavelength']).astype(float)

state_path = str(env.path("examples", "image_cube", "medium", "output", "ang20170323t202244_recon_subs_state"))
reconstruct_subs(
    input_subs_path  = str(env.path("examples", "image_cube", "medium", "output", "ang20170323t202244_subs_state")),
    output_path      = state_path,
    lbl_working_path = str(env.path("examples", "image_cube", "medium", "output", "ang20170323t202244_lbl")),
)

state = envi.open(envi_header(str(state_path)))
state_im = state.open_memmap(interleave='bip')

fig, axs = plt.subplots(1, 4, sharex=True, sharey=True, figsize=(8, 5), tight_layout=True)
axs[0].imshow(normalize(rdn_im[0:400, :, bands], 0, 15))
axs[1].imshow(normalize(state_im[0:400, :, bands], 0, 0.25))
aod = axs[2].imshow(state_im[0:400, :, -2])
h2o = axs[3].imshow(state_im[0:400, :, -1])

plt.colorbar(aod)
plt.colorbar(h2o)

axs[0].set_title('Radiance')
axs[1].set_title('Reflectance')
axs[2].set_title('AOD')
axs[3].set_title('H2O')
plt.show()

Finally, we can examine the final results at the end of the anlaytical line algorithm:

In [ ]:
rfl_path = env.path("examples", "image_cube", "medium", "output", "ang20170323t202244_rfl")
atm_path = env.path("examples", "image_cube", "medium", "output", "ang20170323t202244_atm_interp")

rfl  = envi.open(envi_header(str(rfl_path)))
atm  = envi.open(envi_header(str(atm_path)))

rfl_im = rfl.open_memmap(interleave='bip')
atm_im = atm.open_memmap(interleave='bip')


fig, axs = plt.subplots(1, 4, sharex=True, sharey=True, figsize=(8, 5), tight_layout=True)
axs[0].imshow(normalize(rdn_im[0:400, :, bands], 0, 15))
axs[1].imshow(normalize(rfl_im[0:400, :, bands], 0, 0.25))
aod = axs[2].imshow(atm_im[0:400, :, -2])
h2o = axs[3].imshow(atm_im[0:400, :, -1])

plt.colorbar(aod)
plt.colorbar(h2o)

axs[0].set_title('Radiance')
axs[1].set_title('Reflectance')
axs[2].set_title('AOD')
axs[3].set_title('H2O')
plt.show()